# Execute the Graph-Based Spatial Cross-Validation experiments from ICMLA21 Paper

## 1 - Initialize libraries

In [13]:
import os
import pandas as pd
import geopandas as gpd
from weka.core import jvm
from src import utils
from src.pipeline import Pipeline
from src.visualization.performance import VizMetrics
from src.visualization.dependence import VizDependence

## 2 - Initialize loggers

In [14]:
utils.initialize_coloredlog()
utils.initialize_rich_tracerback()
utils.initialize_logging()

## 3 - Initialize working path and enviromental variables

In [15]:
# Project path
project_dir = os.path.abspath('')[:-5]
# Load enviromental variables
env_var = utils.load_env_variables(project_dir)

## 4 - Set pipeline switchers, the default is to set True to all processes

In [16]:
# Set pipeline switchers
SWITCHERS = {
    "scv": False,
    "fs": False,
    "train": True,
    "predict": True,
    "evaluate": False,
}

# 4 - List all datasets

In [17]:
brazil_removed_datasets = ["Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_north",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_northeast",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_south",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_southeast",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.1_removed_centerwest"]

brazil_datasets = ["Brazil_Election_2018_Sampled_dec0.3_prob0.5",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.6",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.7",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.8",
                   "Brazil_Election_2018_Sampled_dec0.3_prob0.9"]
    

brazil_geoeconomical_regions = ["Brazil_Election_2018_removed_AMAZONIA",
                                "Brazil_Election_2018_removed_NORDESTE",
                                "Brazil_Election_2018_removed_CENTRO_SUL"]

us_datasets = ["US_Corn_Yield_2016", "US_Wheat_Yield_2014"]
just_for_test = ["Brazil_Election_2018_removed_CENTRO_SUL"]

## 6 - Runs the pipeline for each method
OBS: The results and files generated from the pipeline execution will be in the created folder Results in the data directory

### 6.1 Traditional SCV
OBS: We the the paramenter fast True so the semivariogram calculation step that can take 24h is skipped. We calculate the removing buffer by considering the 27 n-degree neighborhood as stated in the paper 

In [18]:
dataset_list = brazil_datasets
fs_method = "CFS"
ml_methods = ["KNN", "OLS", "Lasso", "Ridge", "ElasticNet", "DT", "LGBM", "RF", "MLP", "SVM"]


if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.start()
# Set paths
meshblocks_filename = "meshblocks.shp"
meshblocks_idx = "code_muni"
data_idx = "INDEX"
env_var["root_path"] = "/home/tpinho/IJGIS/Datasets/"
for dataset in dataset_list:
    data_path = os.path.join(env_var["root_path"], dataset, "data.csv")
    
    meshblocks_path = os.path.join(env_var["root_path"], dataset, "meshblocks", meshblocks_filename)
    # Load data
    data = pd.read_csv(data_path, index_col=data_idx, low_memory=False)
    try:
        data.drop(columns=["[GEO]_LATITUDE", "[GEO]_LONGITUDE"], inplace=True)
    except KeyError:
        pass
    meshblocks = gpd.read_file(meshblocks_path)
    # Set meshblocks  index
    meshblocks.set_index(meshblocks_idx, inplace=True)
    for ml_method in ml_methods:
        # Run pipeline
        TraditionalSCV = Pipeline(
            root_path=os.path.join(env_var["root_path"], dataset),
            data=data,
            meshblocks=meshblocks,
            index_col="INDEX",
            fold_col="INDEX_FOLDS",
            target_col="TARGET",
            scv_method="TraditionalSCV",
            fs_method=fs_method,
            ml_method=ml_method,
            switchers=SWITCHERS
        )

        print(f"Running the Traditional SCV approach for dataset: {dataset} ML Method = {ml_method}")
        TraditionalSCV.run()
if fs_method == "CFS" and SWITCHERS["fs"]:
    jvm.stop()

Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.5 ML Method = KNN


Predicting test set: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 69.14it/s]


Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.5 ML Method = OLS


Predicting test set: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 76.82it/s]


Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.5 ML Method = Lasso


...

Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.5 ML Method = SVM


Predicting test set: 100%|████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [00:00<00:00, 90.43it/s]


Running the Traditional SCV approach for dataset: Brazil_Election_2018_Sampled_dec0.3_prob0.6 ML Method = KNN


Training model:  48%|██████████████████████████████████████████████▋                                                  | 13/27 [00:00<00:00, 50.77it/s]


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_20117/1669450797.py:42 in <cell line: 13>                                  │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_20117/1669450797.py'                 │
│ /home/tpinho/IJGIS/Graph-Based_Spatial_Cross_Validation/src/pipeline.py:179 in run        │
│                                                                                           │
│   176 │   │   """Run pipeline"""                                                          │
│   177 │   │   self.generate_pipeline()                                                    │
│   178 │   │   for process in self.pipeline:                                               │
│ ❱ 179 │   │   │   process.run()                                                           │
│   180                                                                                     │
│                                                                                           │
│ /home/tpinho/IJGIS/Graph-Based_Spatial_Cross_Validation/src/model/train.py:169 in run     │
│                                                                                           │
│   166 │   │   for fold in tqdm(folds_name, desc="Training model"):                        │
│   167 │   │   │   params = {}                                                             │
│   168 │   │   │   self._read_train_data(os.path.join(folds_path, fold), data)             │
│ ❱ 169 │   │   │   self._selected_features_filtering(os.path.join(fs_path, f"{fold}.json") │
│   170 │   │   │   model = self._get_model(params=params)                                  │
│   171 │   │   │   model = self._fit(model)                                                │
│   172 │   │   │   self.save_model(model, fold)                                            │
│                                                                                           │
│ /home/tpinho/IJGIS/Graph-Based_Spatial_Cross_Validation/src/model/train.py:67 in          │
│ _selected_features_filtering                                                              │
│                                                                                           │
│    64 │                                                                                   │
│    65 │   def _selected_features_filtering(self, json_path):                              │
│    66 │   │   """Filter only the features selected"""                                     │
│ ❱  67 │   │   selected_features = utils.load_json(json_path)                              │
│    68 │   │   selected_features["selected_features"].append(self.target_col)              │
│    69 │   │   self.train_data = self.train_data[selected_features["selected_features"]]   │
│    70                                                                                     │
│                                                                                           │
│ /home/tpinho/IJGIS/Graph-Based_Spatial_Cross_Validation/src/utils.py:42 in load_json      │
│                                                                                           │
│   39                                                                                      │
│   40 def load_json(json_path):                                                            │
│   41 │   """Load json file."""                                                            │
│ ❱ 42 │   with open(json_path, encoding="utf-8") as file:                                  │
│   43 │   │   return json.load(file)                                                       │
│   44                                                                                      │
╰───────────────────────────────────────────────────